In [1]:
import json
import time

from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware

from src.receipt_log_handler import EthReceiptLogHandler

In [2]:
web3 = Web3(HTTPProvider("https://bsc-dataseed1.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

contract_address = web3.to_checksum_address('0xd50Cf00b6e600Dd036Ba8eF475677d816d6c4281')

with open('abi/lending_pool_aave_v2.json', 'r') as f:
    abi = json.loads(f.read())

contract = web3.eth.contract(abi=abi, address=contract_address)
handler = EthReceiptLogHandler()

### Exercise 1
Crawl dữ liệu event theo 5 loại: Deposit, Borrow, Withdraw, Repay và Liquidate trong 100.000 blocks gần nhất.

In [3]:
end_block = web3.eth.block_number
# 1000 latest blocks
start_block = end_block - 99999
batch_size = 20000

event_types = {'Deposit', 'Borrow', 'Withdraw', 'Repay', 'LiquidationCall'}

event_abi = []
for _abi in abi:
    if (_abi['type'] == 'event') and (_abi['name'] in event_types):
        event_abi.append(_abi)

In [4]:
event_abi_info = handler.build_list_info_event(event_abi)

event_hashes = []
event_subscriber = {}
for event_info in event_abi_info:
    event_hashes.append(event_info[1])
    event_subscriber[event_info[1]] = event_info[0]

In [6]:
event_list = []
from_block = start_block

while from_block < end_block:
    to_block = from_block + batch_size - 1
    if to_block > end_block:
        to_block = end_block

    filter_params = {
        "fromBlock": from_block,
        "toBlock": to_block,
        "topics": [event_hashes],
        "address": [contract_address]
    }

    event_filter = web3.eth.filter(filter_params)
    try:
        events = event_filter.get_all_entries()
    
        for event in events:
            log = handler.web3_dict_to_receipt_log(event)
            eth_event = handler.extract_event_from_log(log, event_subscriber[log.topics[0]])
            if eth_event is not None:
                eth_event_dict = handler.eth_event_to_dict(eth_event)
                event_list.append(eth_event_dict)
    
        print(f'Crawled events from block {from_block} to block {to_block}')
        web3.eth.uninstall_filter(event_filter.filter_id)
        from_block = to_block + 1
    except Exception as e:
            if str(e) == "{'code': -32005, 'message': 'limit exceeded'}":
                print(f"Get err: {str(e)}! Sleep for 5 seconds...!")
                time.sleep(5)
                pass
            else:
                raise e
print(f'Done! Total {len(event_list)} events.')
result = {
    'blocksInfo': {
        'startBlock': start_block,
        'endBlock': end_block
    },
    'event': event_list
}

with open('result/aave1.json', 'w') as f:
    result = json.dumps(result, indent=2)
    f.write(result)

Crawled events from block 36622054 to block 36642053
Get err: {'code': -32005, 'message': 'limit exceeded'}! Sleep for 5 seconds...!
Crawled events from block 36642054 to block 36662053
Get err: {'code': -32005, 'message': 'limit exceeded'}! Sleep for 5 seconds...!
Crawled events from block 36662054 to block 36682053
Crawled events from block 36682054 to block 36702053
Crawled events from block 36702054 to block 36722053
Done! Total 2327 events.


### Exercise 2
Crawl dữ liệu transaction của những event đã crawl.

In [9]:
transaction_hashes = set([event.get('transaction_hash') for event in event_list])

count = 0
transactions = []
begin_time = time.perf_counter()

for tx_hash in transaction_hashes:
    tx = web3.eth.get_transaction(tx_hash)
    tx = json.loads(web3.to_json(tx))
    transactions.append(tx)
    count += 1
    if count == 10:
        print(f'Crawling {count} transactions takes {round(time.perf_counter() - begin_time,2)} seconds.')
        
        begin_time = time.perf_counter()
        count = 0

print(f'Done! Total {len(transactions)} transactions')

with open('result/aave2.json', 'w') as f:
    result = json.dumps({
        'blocksInfo': {
            'startBlock': start_block,
            'endBlock': end_block
        },
        'transactions': transactions
    }, indent=2)
    f.write(result)


Done! Total 1 transactions


### Exercise 3
Lấy thông tin địa chỉ token được thực hiện giao dịch Deposit và Borrow nhiều nhất (gợi ý: địa chỉ token nằm trong trường reserve trong event)

In [10]:
reserves_freq = {}
for event in event_list:
    if event.get('event_type') not in {'BORROW', 'DEPOSIT'}:
        continue

    reserve = event.get('reserve')
    if reserve not in reserves_freq:
        reserves_freq[reserve] = 1
    else:
        reserves_freq[reserve] += 1

max_freq = max(reserves_freq.values())
most_reserved_tokens = [token for token, frequency in reserves_freq.items() if frequency == max_freq]

result = {
    'blocksInfo': {
        'startBlock': start_block,
        'endBlock': end_block
    },
    'maxFrequency': max_freq,
    'tokens': most_reserved_tokens
}

print(f'max frequency: {max_freq}')
print(f'tokens: {most_reserved_tokens}')

with open('result/aave3.json', 'w') as f:
    result = json.dumps(result, indent=2)
    f.write(result)

max frequency: 467
tokens: ['0x55d398326f99059ff775485246999027b3197955']


### Exercise 4
Lấy thông tin địa chỉ thực hiện transaction nhiều nhất trong lending pool (gợi ý: địa chỉ ví nằm trong trường from của dữ liệu transaction, nếu số transaction của các địa chỉ bằng nhau thì lấy địa chỉ bất kỳ).

In [11]:
wallet_addresses = [tx.get('from') for tx in transactions]

wallet_address_freq = {}
for address in wallet_addresses:
    if address not in wallet_address_freq:
        wallet_address_freq[address] = 1
    else:
        wallet_address_freq[address] += 1

max_frequency = max(wallet_address_freq.values())
most_active_wallets = [wallet for wallet, frequency in wallet_address_freq.items() if frequency == max_frequency]

wallets_info = []
for wallet in most_active_wallets:
    wallet_info = contract.functions.getUserAccountData(web3.to_checksum_address(wallet)).call()
    wallets_info.append({
        'address': wallet,
        'totalCollateralETH': wallet_info[0],
        'totalDebtETH': wallet_info[1],
        'availableBorrowsETH': wallet_info[2],
        'currentLiquidationThreshold': wallet_info[3],
        'ltv': wallet_info[4],
        'healthFactor': wallet_info[5],
    })

    """
    totalCollateralETH:          the total collateral in ETH of the user
    totalDebtETH:                the total debt in ETH of the user
    availableBorrowsETH:         the borrowing power left of the user
    currentLiquidationThreshold: the liquidation threshold of the user
    ltv:                         the loan to value of the user
    healthFactor:                the current health factor of the user 
    """
    
result = {
    'blocksInfo': {
        'startBlock': start_block,
        'endBlock': end_block
    },
    'maxFrequency': max_frequency,
    'wallets': wallets_info
}

print(f'max frequency: {max_frequency}')
print(f'wallets: {wallets_info}')

with open('result/aave4.json', 'w') as f:
    result = json.dumps(result, indent=2)
    f.write(result)

max frequency: 1
wallets: [{'address': '0xB53ED4b7B068d41b590927650f4dFB2F797C3b22', 'totalCollateralETH': 12739422302499, 'totalDebtETH': 8153505103068, 'availableBorrowsETH': 764090508681, 'currentLiquidationThreshold': 7500, 'ltv': 7000, 'healthFactor': 1171835499713958441}]


### Exercise 5
Dựa vào địa chỉ Oracle (Aave oracle) và abi/oracle_abi.json tìm giá (asset price) của token ở câu 3.

In [14]:
oracle_address = web3.to_checksum_address('0x3436c4B4A27B793539844090e271591cbCb0303c')
token = web3.to_checksum_address(most_reserved_tokens[0])
BASE18 = 10 ** 18

with open('abi/oracle_abi.json', 'r') as f:
    oracle_abi = json.loads(f.read())

oracle_contract = web3.eth.contract(abi=oracle_abi, address=oracle_address)
token_price = oracle_contract.functions.getAssetPrice(token).call() / BASE18

print(f'token: {token}')
print(f'token price: {token_price}')

with open('result/aave5.json', 'w') as f:
    result = json.dumps({
        'token': token,
        'price': token_price
    }, indent=2)
    f.write(result)

token: 0x55d398326f99059fF775485246999027B3197955
token price: 1.00057221
